In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import warnings
warnings.filterwarnings("ignore")

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
knn = KNeighborsClassifier(n_neighbors = 3)
dtc = DecisionTreeClassifier()
rfc =RandomForestClassifier()
Lr = LogisticRegression()
gnb = GaussianNB()
mnb = MultinomialNB()
svc = SVC()
xgblog = xgb.XGBClassifier(objective = "binary:logistic")
xgblogr = xgb.XGBClassifier(objective = "binary:logitraw")
xgbhinge = xgb.XGBClassifier(objective = "binary:hinge")

clfs = [dtc,rfc,Lr,gnb,svc,xgblog,xgblogr,xgbhinge,knn]
scores = []

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot

train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
data = pd.concat([train,test],axis=0)
datas = [train,test]
data.shape


(1309, 12)

In [3]:
train.Embarked = train[["Embarked"]].fillna(data.Embarked.mode()[0])
train.Age = train.Age.fillna(data.Age.mean())
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
train = train.drop(["PassengerId","Name","Ticket","Cabin"],axis=1)
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [5]:
Y = train[["Survived"]]
train = train.drop(["Survived"],axis=1)

Y.shape
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [6]:
train.Sex = (train.Sex == "male").astype(int)
train.Sex

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [7]:
train["Embarked"] = train["Embarked"].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
train.Embarked

0      0
1      1
2      0
3      0
4      0
      ..
886    0
887    0
888    0
889    1
890    2
Name: Embarked, Length: 891, dtype: int64

In [8]:
train["FamilySize"] = train["Parch"] + train["SibSp"] + 1 
train.tail()
train['IsAlone'] = 0
train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1
train





train = train.drop(["Parch","SibSp"],axis=1)
train

,Pclass,Sex,Age,Fare,Embarked,FamilySize,IsAlone
0,3,1,22.000000,7.2500,0,2,0
1,1,0,38.000000,71.2833,1,2,0
2,3,0,26.000000,7.9250,0,1,1
3,1,0,35.000000,53.1000,0,2,0
4,3,1,35.000000,8.0500,0,1,1
...,...,...,...,...,...,...,...
886,2,1,27.000000,13.0000,0,1,1
887,1,0,19.000000,30.0000,0,1,1
888,3,0,29.881138,23.4500,0,4,0
889,1,1,26.000000,30.0000,1,1,1


In [9]:

import sklearn.preprocessing as pr
ohe = pr.OneHotEncoder()
emb = train["Embarked"]
print(emb.unique())
emb = ohe.fit_transform(train[["Embarked"]]).toarray()
train = train.drop(["Embarked"],axis=1)
emb = pd.DataFrame(emb)
train = pd.concat([emb.iloc[:,0:2],train],axis=1)
train.head()


[0 1 2]


,0,1,Pclass,Sex,Age,Fare,FamilySize,IsAlone
0,1.0,0.0,3,1,22.0,7.2500,2,0
1,0.0,1.0,1,0,38.0,71.2833,2,0
2,1.0,0.0,3,0,26.0,7.9250,1,1
3,1.0,0.0,1,0,35.0,53.1000,2,0
4,1.0,0.0,3,1,35.0,8.0500,1,1


In [10]:
#Modelin eğitimi ve değerlendirmesi
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(train,Y,
                                             random_state = 42,
                                             test_size = 0.3)




In [11]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
ss = StandardScaler()
mms = MinMaxScaler()

In [12]:
for clf in clfs:
    clf.fit(ss.fit_transform(xtrain),ytrain)
    score = clf.score(ss.fit_transform(xtest),ytest)
    scores.append(score)
scores 

[12:34:39] WARNING: ../src/learner.cc:1205: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[0.753731343283582,
 0.7798507462686567,
 0.8059701492537313,
 0.7761194029850746,
 0.8022388059701493,
 0.8059701492537313,
 0.7910447761194029,
 0.7611940298507462,
 0.7873134328358209]

In [13]:
"""
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(ss.fit_transform(xtrain),ytrain)

print(grid.best_params_)
print(grid.best_estimator_)
"""

"\nfrom sklearn.model_selection import GridSearchCV\n  \n# defining parameter range\nparam_grid = {'C': [0.1, 1, 10, 100, 1000], \n              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],\n              'kernel': ['rbf']} \n  \ngrid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)\n  \n# fitting the model for grid search\ngrid.fit(ss.fit_transform(xtrain),ytrain)\n\nprint(grid.best_params_)\nprint(grid.best_estimator_)\n"

In [14]:
"""
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(xtrain,ytrain)

print(grid.best_params_)
print(grid.best_estimator_)
"""

"\nfrom sklearn.model_selection import GridSearchCV\n  \n# defining parameter range\nparam_grid = {'C': [0.1, 1, 10, 100, 1000], \n              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],\n              'kernel': ['rbf']} \n  \ngrid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)\n  \n# fitting the model for grid search\ngrid.fit(xtrain,ytrain)\n\nprint(grid.best_params_)\nprint(grid.best_estimator_)\n"

In [15]:

#Test verisinin hazırlanması
test = test.drop(["PassengerId","Name","Ticket","Cabin"],axis=1)
test.Embarked = test[["Embarked"]].fillna(data.Embarked.mode()[0])
test.Fare = test[["Fare"]].fillna(data.Fare.mean())
test.Age = test[["Age"]].fillna(data.Age.mean())
test.Sex = (test.Sex == "male").astype(int)


test_emb = test[["Embarked"]]
test_emb = ohe.fit_transform(test_emb).toarray()
test_emb = pd.DataFrame(test_emb)
test = pd.concat([test_emb.iloc[:,0:2],test],axis = 1)
test = test.drop(["Embarked"],axis = 1)



test["FamilySize"] = test["Parch"] + test["SibSp"] + 1 
test.tail()
test['IsAlone'] = 0
test.loc[test['FamilySize'] == 1, 'IsAlone'] = 1
test = test.drop(["Parch","SibSp"],axis=1)


test

,0,1,Pclass,Sex,Age,Fare,FamilySize,IsAlone
0,0.0,1.0,3,1,34.500000,7.8292,1,1
1,0.0,0.0,3,0,47.000000,7.0000,2,0
2,0.0,1.0,2,1,62.000000,9.6875,1,1
3,0.0,0.0,3,1,27.000000,8.6625,1,1
4,0.0,0.0,3,0,22.000000,12.2875,3,0
...,...,...,...,...,...,...,...,...
413,0.0,0.0,3,1,29.881138,8.0500,1,1
414,1.0,0.0,1,0,39.000000,108.9000,1,1
415,0.0,0.0,3,1,38.500000,7.2500,1,1
416,0.0,0.0,3,1,29.881138,8.0500,1,1


In [16]:
knn1 = KNeighborsClassifier(n_neighbors = 3)
dtc1 = DecisionTreeClassifier()
rfc1 =RandomForestClassifier()
Lr1 = LogisticRegression()
gnb1 = GaussianNB()
mnb1 = MultinomialNB()
svc1 = SVC()
gnb1.fit(mms.fit_transform(xtrain),ytrain)
tahmin = gnb.predict(mms.fit_transform(test))

tahmin = pd.DataFrame(tahmin)
#tahmin.columns = ["Survived"]
index = []
for i in range(892,1310):
    index.append(i)
index = pd.DataFrame(index)
tahmin = pd.concat([index,tahmin],axis=1)
tahmin.columns = ["PassengerId","Survived"]
tahmin
tahmin.to_csv("submission.csv",index = False)

